<a href="https://colab.research.google.com/github/PraveenSingh-ML-0728/Classification_Problems/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#while True:pass

Importing Dependencies

In [24]:
import numpy as np
import pandas as pd
import re     #RegularExpression:-For searching a text in a Documents ...
from nltk.corpus import stopwords   #NaturalLanguageToolKit(stopwords: the that a an or we say article...)
from nltk.stem.porter import PorterStemmer  #Stemming is the process of reducing a word to its Root Word ... Example :- actor, actress, acting ---> act
from sklearn.feature_extraction.text import TfidfVectorizer   #TfidfVectorizer is for converting the text into features vector...
from sklearn.model_selection import train_test_split    #Use to split our Data into training and Test Data ...
from sklearn.linear_model import LogisticRegression     #Select the LogisticRegression Model ...
from sklearn.metrics import accuracy_score    #To find the Accuracy of the Model ...

In [25]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
#Printing the stop words in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [27]:
news_dataset =pd.read_csv('/content/train.csv')

In [29]:
news_dataset.shape  #20800 Articles and 5 Features

(20800, 5)

In [30]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
#Counting the number of missing values in datasets
news_dataset.isnull().sum() #There are 558 missing value in title column in text 39 are missing etc ...

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [34]:
#Replacing the null values by string
news_dataset = news_dataset.fillna('')

In [35]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [36]:
# Now merging the title and Author name as because its seem a important feature here ...
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [37]:
print(news_dataset['content'])  #Here we can use the new content column for Prediction

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [38]:
# Now Separating the Data and Labels
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [39]:
print(X)
print(Y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns]
0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

Stemming:

Stemming is the process of reducing a word to its Root Word ...
Example :- actor, actress, acting ---> act

In [40]:
port_stem = PorterStemmer()

In [42]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #remove all the numbers and puncuation in a content ...and the numbers and puncuation are replaced by a space ...
  stemmed_content = stemmed_content.lower() #convert all the letters to lowercase ...
  stemmed_content = stemmed_content.split() #All words get splitted and converted to a list ...
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # This means in my content I would take all the words which are not a stopwords ...
  stemmed_content = ' '.join(stemmed_content) #Now Joining all the words having space in Between ...
  return stemmed_content  #Then return with our Processed Text as we get the text having No numbers puncuations etc ...

In [43]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [44]:
print(news_dataset['content'])  #Here we are getting all word in Lowercase  and no numbers and puncuation ...

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [49]:
#Separating the Data and Label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [50]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [51]:
print(Y)

[1 0 1 ... 0 1 1]


In [55]:
Y.shape

(20800,)

In [56]:
# Converting the textual data to numerical Data
vectorizer = TfidfVectorizer()  #TermfrequencyInverseDocumentFrequency:- counts the no.of times a particular word is repeating in a paragraph or text so it is very important as because it always repeats so it assign a numeric value to that repeated word ...
vectorizer.fit(X)

X = vectorizer.transform(X)

In [57]:
print(X)  # Now it converted to Numeric value by TfidfVectorizer as because our machine learning Model 
# can't understand the textual data or string so that's why we transform the text to numeric values for good understanding for our ML Model ...

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting our Dataset to Training & Test Data

In [59]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size =0.2, stratify=Y, random_state=2)

Training The Model: LogisticRegression

In [60]:
model = LogisticRegression()

In [61]:
model.fit(X_train, Y_train) #Trained our Model ...

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Evaluation

Now will find the Accuracy value/Score

In [62]:
#Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train) #Y_train is the original label and 
                                                                      #X_train_prediction is predicted by our trained model ...

In [63]:
print('Accuracy Score Of The Training Data : ',training_data_accuracy)

Accuracy Score Of The Training Data :  0.9865985576923076


In [64]:
#Accuracy score on the test data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [65]:
print('Accuracy Score Of The Testing Data : ',testing_data_accuracy)

Accuracy Score Of The Testing Data :  0.9790865384615385


Making A Predictive System ...

In [72]:
X_new = X_test[1000] # 0,1,2 ... means the first/second/thire ... row in our X_test column ...

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("The news is Real ...")
else:
  print("The news is Fake ...")

[0]
The news is Real ...


In [73]:
#For testing our model is predicting correct or not we can test it with our Y_test data...
print(Y_test[1000])

0
